### Actividad: Repaso de conceptos iniciales

####  Creación y ejecución de hilos, una forma común de implementar concurrencia

In [ ]:
import threading

# Definición de una función para el hilo
def cocinar_plato(nombre):
    print(f"Cocinando {nombre}...")

# Creación de hilos para diferentes platos
hilo1 = threading.Thread(target=cocinar_plato, args=("Paella",))
hilo2 = threading.Thread(target=cocinar_plato, args=("Ratatouille",))

# Inicio de los hilos
hilo1.start()
hilo2.start()

# Esperar a que ambos hilos terminen
hilo1.join()
hilo2.join()

print("Todos los platos están listos!")

Este código es como tener dos chefs en la cocina: uno preparando `Paella` y el otro `Ratatouille`. Ambos trabajan al mismo tiempo, cada uno en su tarea, demostrando la esencia de la concurrencia. La coordinación se maneja a través del uso de `join()`, asegurando que el programa principal espere a que ambos chefs (hilos) terminen antes de declarar que todos los platos están listos.



#### Uso de threading y un recurso compartido

In [ ]:
import threading

# Variable compartida
plato_en_preparación = 0

# Función para modificar el recurso compartido
def preparar_plato():
    global plato_en_preparación
    for _ in range(1000000):
        plato_en_preparación += 1

# Crear dos hilos que modifican el recurso compartido
hilo1 = threading.Thread(target=preparar_plato)
hilo2 = threading.Thread(target=preparar_plato)

hilo1.start()
hilo2.start()

hilo1.join()
hilo2.join()

print(f"Total de platos preparados: {plato_en_preparación}")


Este ejemplo ilustra cómo la falta de sincronización puede llevar a un recuento incorrecto de platos preparados, subrayando la importancia de mecanismos como semáforos o bloqueos para gestionar el acceso al recurso compartido.

#### Simulación del paso de mensajes usando colas

In [ ]:
import threading
import queue

# Cola para mensajes
orden_cocina = queue.Queue()

# Función del chef que solicita ingredientes
def chef():
    for ingrediente in ["tomate", "queso", "masa"]:
        print(f"Chef: necesito {ingrediente}")
        orden_cocina.put(ingrediente)

# Función del ayudante que provee ingredientes
def ayudante():
    while not orden_cocina.empty():
        ingrediente = orden_cocina.get()
        print(f"Ayudante: aquí tienes {ingrediente}")

# Iniciar los hilos
threading.Thread(target=chef).start()
threading.Thread(target=ayudante).start()


Este código representa una cocina donde la comunicación clara y la división de tareas previenen el caos, demostrando cómo el paso de mensajes facilita la colaboración sin conflictos directos sobre los recursos.

#### Creación de procesos y threads para realizar tareas simples

In [ ]:
import threading
import multiprocessing

# Tarea para threads
def tarea_thread(nombre):
    print(f"Thread {nombre} está corriendo")

# Tarea para procesos
def tarea_proceso(nombre):
    print(f"Proceso {nombre} está corriendo")

# Crear y ejecutar threads
threads = [threading.Thread(target=tarea_thread, args=(f"Thread-{i}",)) for i in range(3)]
for t in threads:
    t.start()
for t in threads:
    t.join()

# Crear y ejecutar procesos
procesos = [multiprocessing.Process(target=tarea_proceso, args=(f"Proceso-{i}",)) for i in range(3)]
for p in procesos:
    p.start()
for p in procesos:
    p.join()


Este ejemplo demuestra cómo crear y ejecutar múltiples threads y procesos en Python. Cada thread dentro de un proceso puede ejecutarse concurrentemente, compartiendo el mismo espacio de memoria, mientras que cada proceso se ejecuta de manera independiente, con su propio espacio de memoria.

#### El concepto de time-slicing 

In [ ]:
import threading
import time

# Definición de las tareas
def tarea1():
    for i in range(5):
        print("Tarea 1 se está ejecutando")
        time.sleep(0.5)  # Simula el trabajo de la tarea

def tarea2():
    for i in range(5):
        print("Tarea 2 se está ejecutando")
        time.sleep(0.5)  # Simula el trabajo de la tarea

# Creación de los threads
hilo1 = threading.Thread(target=tarea1)
hilo2 = threading.Thread(target=tarea2)

# Inicio de los threads
hilo1.start()
hilo2.start()

# Esperamos a que ambos hilos terminen
hilo1.join()
hilo2.join()


Este ejemplo demuestra cómo dos tareas (simuladas por `tarea1` y `tarea2`) pueden "compartir" el CPU, alternando su ejecución mediante el uso de pausas (`time.sleep`), que simulan el trabajo realizado. Aunque este código no implementa `time-slicing` directamente (ya que ese es un mecanismo gestionado por el sistema operativo), ayuda a visualizar cómo diferentes tareas pueden avanzar aparentemente de manera simultánea.

#### Sincronizando acceso a memoria compartida

In [ ]:
import threading

# Un contador compartido
contador = 0

# Un lock para sincronizar el acceso al contador
lock = threading.Lock()

def incrementar_contador():
    global contador
    for _ in range(100000):
        with lock:
            contador += 1

# Creando threads para incrementar el contador
threads = [threading.Thread(target=incrementar_contador) for _ in range(10)]

# Iniciar threads
for t in threads:
    t.start()

# Esperar a que todos los threads terminen
for t in threads:
    t.join()

print(f"Valor final del contador: {contador}")


Ejemplo de cómo varios threads pueden modificar de forma segura un objeto compartido en Python utilizando `Lock` para sincronizar el acceso. Este código demuestra un patrón clásico en la programación concurrente: sincronizar el acceso a recursos compartidos (en este caso, un contador simple) para evitar condiciones de carrera. Aquí, `Lock` actúa como un vigilante, asegurando que solo un thread a la vez pueda modificar el contador. Esto evita que los threads sobrescriban los cambios de los demás, garantizando que el contador se incremente correctamente.

#### Condiciones de carrera

In [ ]:
import threading

# Recurso compartido
contador = 0

def incrementar():
    global contador
    for _ in range(10000):
        contador += 1

def decrementar():
    global contador
    for _ in range(10000):
        contador -= 1

# Creando threads
thread1 = threading.Thread(target=incrementar)
thread2 = threading.Thread(target=decrementar)

# Iniciando threads
thread1.start()
thread2.start()

# Esperando a que ambos threads terminen
thread1.join()
thread2.join()

print(f"Valor final del contador: {contador}")


En este código, dos threads modifican un contador compartido: uno lo incrementa y el otro lo decrementa. Sin sincronización adecuada, las operaciones de incremento y decremento pueden entrelazarse de manera que el contador final no sea cero como se esperaría, revelando una condición de carrera.

#### División de una tarea sencilla entre varios procesos en Python

In [ ]:
from concurrent.futures import ProcessPoolExecutor
import math

# Una tarea simple que queremos ejecutar en paralelo
def calcular_raiz_cuadrada(numeros):
    return {n: math.sqrt(n) for n in numeros}

# Dividir una lista de números en 4 partes para procesar en paralelo
numeros = range(1, 1001)
chunk_size = len(numeros) // 4
chunks = [numeros[i:i + chunk_size] for i in range(0, len(numeros), chunk_size)]

# Usar ProcessPoolExecutor para ejecutar nuestras tareas en paralelo
with ProcessPoolExecutor(max_workers=4) as executor:
    resultados = list(executor.map(calcular_raiz_cuadrada, chunks))

# Fusionar los resultados de los chunks en un solo diccionario
resultados_finales = {k: v for d in resultados for k, v in d.items()}

print(f"Algunos resultados: {list(resultados_finales.items())[:10]}")


Este ejemplo demuestra cómo podemos dividir un problema (calcular la raíz cuadrada de una lista de números) en partes más pequeñas y ejecutar esas partes en paralelo, aprovechando múltiples núcleos de procesamiento. Al hacerlo, completamos la tarea general más rápidamente de lo que podríamos secuencialmente.

#### Simulación de la arquitectura MIMD

In [ ]:
from multiprocessing import Process, Queue
import os

def calcular_cuadrados(numeros, resultados):
    print(f"Proceso {os.getpid()} calculando cuadrados.")
    for n in numeros:
        resultados.put(n * n)

def calcular_cubos(numeros, resultados):
    print(f"Proceso {os.getpid()} calculando cubos.")
    for n in numeros:
        resultados.put(n * n * n)

if __name__ == "__main__":
    numeros = range(5)
    resultados_cuadrados = Queue()
    resultados_cubos = Queue()

    proceso_cuadrados = Process(target=calcular_cuadrados, args=(numeros, resultados_cuadrados))
    proceso_cubos = Process(target=calcular_cubos, args=(numeros, resultados_cubos))

    proceso_cuadrados.start()
    proceso_cubos.start()

    proceso_cuadrados.join()
    proceso_cubos.join()

    while not resultados_cuadrados.empty():
        print(f"Cuadrado: {resultados_cuadrados.get()}")

    while not resultados_cubos.empty():
        print(f"Cubo: {resultados_cubos.get()}")


Este script ilustra el concepto de MIMD, ejecutando dos tareas distintas (calcular cuadrados y cubos) en paralelo, cada una en su propio proceso. A través de este enfoque, podemos ver cómo diferentes instrucciones operan sobre diferentes conjuntos de datos, un eco de la flexibilidad y potencia que las arquitecturas MIMD ofrecen en el mundo real de la computación paralela.

#### Simulando la paralelización automática

In [ ]:
from concurrent.futures import ThreadPoolExecutor
import time

# Función que simula una tarea que consume tiempo
def tarea_lenta(n):
    print(f"Iniciando tarea {n}")
    time.sleep(2)  # Simula tiempo de procesamiento
    print(f"Tarea {n} completada")
    return f"Resultado de tarea {n}"

# Ejecución secuencial
inicio_sec = time.time()
for i in range(3):
    resultado = tarea_lenta(i)
    print(resultado)
fin_sec = time.time()
print(f"Tiempo de ejecución secuencial: {fin_sec - inicio_sec} segundos")

# Ejecución paralela
inicio_par = time.time()
with ThreadPoolExecutor(max_workers=3) as executor:
    resultados = list(executor.map(tarea_lenta, range(3)))
    for resultado in resultados:
        print(resultado)
fin_par = time.time()
print(f"Tiempo de ejecución paralela: {fin_par - inicio_par} segundos")


En este ejemplo, la tarea de ejecución secuencial que tardaría 6 segundos (simulando tres tareas que tardan 2 segundos cada una), se completa en aproximadamente 2 segundos con la ejecución paralela, mostrando el potencial de la paralelización para mejorar la eficiencia.

#### Uso de multiprocessing para paralelismo

In [ ]:
from multiprocessing import Pool

# Una función que modela una tarea computacionalmente intensiva
def tarea_intensiva(n):
    sum([i*n for i in range(10000)])
    return n

# Lista de números para procesar
numeros = range(10)

# Usar un Pool de procesos para ejecutar nuestras tareas en paralelo
if __name__ == '__main__':
    with Pool(5) as p:
        print(p.map(tarea_intensiva, numeros))


#### Simulación de procesamiento paralelo


In [ ]:
from concurrent.futures import ThreadPoolExecutor
import time

def tarea_simulada(duracion):
    print(f"Iniciando tarea que tarda {duracion} segundos")
    time.sleep(duracion)
    return f"Tarea que tarda {duracion} segundos completada"

# Lista de duraciones para cada tarea
duracion_de_tareas = [1, 3, 2, 4]

# Ejecutar tareas en paralelo
with ThreadPoolExecutor(max_workers=4) as executor:
    resultados = list(executor.map(tarea_simulada, duracion_de_tareas))

for resultado in resultados:
    print(resultado)


Este script simula la ejecución de varias tareas con diferentes duraciones en paralelo, utilizando un pool de threads. Cada tarea representa un cálculo o proceso que se puede realizar simultáneamente, reflejando el concepto de procesamiento paralelo dentro del contexto más amplio de la computación paralela.

#### Computación secuencial vs. paralela

In [ ]:
import time
from multiprocessing import Pool

# Tarea simulada para demostrar tiempo de procesamiento
def tarea_simulada(duracion):
    time.sleep(duracion)
    return duracion

# Lista de tareas con duraciones simuladas
tareas = [1, 2, 3, 4]

# Computación secuencial
inicio_sec = time.time()
for tarea in tareas:
    tarea_simulada(tarea)
fin_sec = time.time()
print(f"Computación secuencial completada en {fin_sec - inicio_sec} segundos.")

# Computación paralela
inicio_par = time.time()
with Pool(4) as p:
    p.map(tarea_simulada, tareas)
fin_par = time.time()
print(f"Computación paralela completada en {fin_par - inicio_par} segundos.")


Este código demuestra cómo el procesamiento paralelo puede reducir significativamente el tiempo total necesario para completar un conjunto de tareas en comparación con un enfoque secuencial, destacando la eficiencia que la computación paralela puede aportar a problemas complejos.

#### Simulando escalabilidad y resiliencia

In [ ]:
from multiprocessing import Process, current_process
import os
import time
import random

def tarea_distribuida():
    print(f"Proceso {current_process().name} iniciado.")
    # Simula tiempo de ejecución con una pausa
    tiempo = random.randint(1, 5)
    time.sleep(tiempo)
    print(f"Proceso {current_process().name} completado en {tiempo} segundos.")

if __name__ == '__main__':
    procesos = []

    # Crear procesos para simular nodos adicionales
    for i in range(5):
        proceso = Process(target=tarea_distribuida, name=f"Nodo_{i+1}")
        procesos.append(proceso)
        proceso.start()

    # Esperar a que todos los procesos terminen
    for proceso in procesos:
        proceso.join()

    print("Todos los nodos han completado sus tareas.")


Este código crea varios procesos que simulan nodos en un sistema distribuido, demostrando cómo se pueden añadir "nodos" para manejar tareas paralelas y cómo la falla (simulada aquí por tiempos de ejecución variables) de un nodo no impide que el sistema complete su trabajo.

### Ejercicios

1. ¿Cuál es la principal diferencia entre la programación concurrente usando memoria compartida y el paso de mensajes?

2. Describe una situación donde el modelo de paso de mensajes sería preferible sobre el modelo de memoria compartida.

3. Explica cómo los threads dentro de un mismo proceso comparten recursos y qué desafíos puede presentar esta compartición.

4. ¿Cómo contribuye el time-slicing a la ilusión de multitarea en sistemas operativos de un solo núcleo?

5. ¿Por qué es importante asegurar que las modificaciones a estructuras de datos compartidas sean atómicas en programación concurrente?

6. ¿Qué hace que las condiciones de carrera sean especialmente difíciles de identificar y corregir en la programación concurrente?

7. Describe cómo la computación paralela puede ser utilizada en el procesamiento de imágenes.

8. Explica la diferencia entre las arquitecturas SIMD y MIMD.

9. ¿Cuáles son las ventajas de utilizar procesadores multi-núcleo para la computación paralela?

10. ¿Cómo afecta el SMP a la visibilidad uniforme de la memoria entre CPUs?

11. Proporciona un ejemplo de cómo la computación distribuida puede ser aplicada en el mundo real.

12. [Numba](https://numba.pydata.org/) es una biblioteca que puede traducir automáticamente un subconjunto de operaciones Python y NumPy a código de máquina rápido. Esto se puede usar para automatizar la paralelización de ciertos cálculos intensivos en datos. Utiliza Numba para paralelizar una operación simple sobre un array de NumPy.
    ```
    pip install numba
  
    from numba import jit, prange
    import numpy as np

    @jit(nopython=True, parallel=True)
        ...
```     



13. Dask es una biblioteca de Python flexible para computación paralela que se adapta bien a los entornos de sistemas distribuidos, especialmente para el análisis de datos a gran escala. Dask permite realizar cálculos paralelos sobre datasets que no caben en la memoria, distribuyéndolos a través de múltiples núcleos o incluso máquinas. 
Completa el código apra  utilizar Dask para paralelizar y distribuir cálculos de manera efectiva.

    Primero, asegúrate de instalar Dask:

    ```
    pip install dask[distributed]  # Instala Dask y las herramientas para sistemas distribuidos
    ```

    Este ejemplo crea un array Dask grande, que se divide en trozos más pequeños (chunks), y luego calcula la suma de todos los elementos del array. Al trabajar con Dask, estos cálculos se pueden distribuir automáticamente a través de los núcleos disponibles en tu máquina, o en un clúster de máquinas, si se configura.

    ```
    from dask.distributed import Client
    import dask.array as da

    def main():
        # Iniciar un cliente Dask. Esto nos conectará a un clúster si existe,
        # o iniciará uno localmente si no hay ninguno.
        client = Client()

        # Crea un array Dask grande, de forma similar a como se haría con NumPy
        # pero especificando el tamaño de 'chunks' para la división en trozos.
        // Completa

        # Calcula la suma del array.
        suma_total = array.sum()

        # Dask utiliza ejecución perezosa, así que necesitamos llamar a .compute()
        # para forzar la ejecución del cálculo.
        resultado = suma_total.compute()

        print(f"La suma total del array es: {resultado}")

    if __name__ == "__main__":
        main()
    ```

#### Completa